In [1]:
import tensorflow as tf
from tensorflow.keras.models import clone_model
from model_utils import simplecnn
import numpy as np

In [2]:
class clint_k():
    def __init__(self, name, dataset, model, 
                optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                ):
        self.name = name
        self.dataset = dataset
        self.attri = {'name':self.name, 'dataset':self.dataset}
        self.model = model 
        self.optimizer = optimizer
        self.model.compile(self.optimizer, loss_fn)
        self.warmingUp = False
        self.loss_fn = loss_fn
    @tf.function
    def set_params(self, weights): # input weight
        self.model.set_weights(weights)
        
#     @tf.function
    def warming_up(self):
        print('Warming up process, setting slot from trainable weight')
        layer_trainable = {}
        model = self.model
        for l in model.layers:
            layer_trainable[l.name] = l.trainable
            l.trainable = False
            
        model.fit(self.dataset, verbose = False)

        for l in layer_trainable.keys():
            model.get_layer(l).trainable = True
#     def warming_up(self):

        # =================
#         for step, (x_batch_train, y_batch_train) in enumerate(self.dataset):

#             # Open a GradientTape to record the operations run
#             # during the forward pass, which enables auto-differentiation.
#             with tf.GradientTape() as tape:

#                 # Run the forward pass of the layer.
#                 # The operations that the layer applies
#                 # to its inputs are going to be recorded
#                 # on the GradientTape.
#                 logits = self.model(x_batch_train, training=True)  # Logits for this minibatch

#                 # Compute the loss value for this minibatch.
#                 loss_value = self.loss_fn(y_batch_train, logits)

#             # Use the gradient tape to automatically retrieve
#             # the gradients of the trainable variables with respect to the loss.
#             grads = tape.gradient(loss_value, model.trainable_weights)

            # Run one step of gradient descent by updating
            # the value of the variables to minimize the loss.
#             self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))

        # ============
#     def warming_up(self):
#         model = self.model
#         model([x for (x,y) in self.dataset.take(1)][0], training=True)

    def local_update(self,
                    epochs = 1
                    ):
        if not self.warmingUp:
            self.warming_up()
            self.optimizer.set_params(self.model)
            self.warmingUp = True
            print('self.warmingUp finish')
        else:
            self.optimizer.set_params(self.model)
        return model.fit(self.dataset, epochs=epochs)
        


In [3]:
class PerturbedGradientDescent(tf.keras.optimizers.SGD):
    def __init__(self,
              name="PGD",
              mu=0.01,
               **kwargs):
        super().__init__(name = name, **kwargs)
#         print(name)

        self._set_hyper("prox_mu", mu)
        # Tensor versions of the constructor arguments, created in _prepare().
        self._lr_t = None
        self._mu_t = None
        self.vstar = None
        
        self._lr_t = tf.convert_to_tensor(self._hyper['learning_rate'], name="learning_rate")
        self._mu_t = tf.convert_to_tensor(self._hyper['prox_mu'], name="prox_mu")
        
    def _create_slots(self, var_list):

        for var in var_list:
            self.add_slot(var, "vt")

    
    def _resource_apply_dense(self, grad, var):
        lr_t = tf.cast(self._lr_t, var.dtype.base_dtype)
        mu_t = tf.cast(self._mu_t, var.dtype.base_dtype)
        
        vp = self.get_slot(var, "vt") # mv = this client's traiable variable 
        
        var_update = var.assign_sub(lr_t*(grad + mu_t*(var-vp)))

        return tf.raw_ops.ResourceApplyGradientDescent(
          var=var_update.handle,
          alpha=lr_t,
          delta=grad,
          use_locking=self._use_locking)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
            "decay": self._initial_decay,
            "momentum": self._serialize_hyperparameter("momentum"),
            "nesterov": self.nesterov,
            "prox_mu": self.mu,
        })
        return config
    
    def set_params(self, model):
        for weights in model.trainable_variables:
            vp = self.get_slot(weights, "vt") 
            vp.assign(weights)
#             print(vp)
            

In [4]:
model = simplecnn('server')
type(model) 

tensorflow.python.keras.engine.functional.Functional

In [5]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test_origin, y_test_origin) = mnist.load_data()

X_train = X_train/255.0

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).cache().batch(64).prefetch(1)

In [7]:
import time
z = clint_k(name='id0', dataset=train_dataset, model=model,
           optimizer = PerturbedGradientDescent())
tf.keras.backend.clear_session()
z.local_update(epochs = 3)
# 

Warming up process, setting slot from trainable weight
self.warmingUp finish
Epoch 1/3
938/938 [==============================] - 2s 3ms/step - loss: 1.7246
Epoch 2/3
938/938 [==============================] - 2s 3ms/step - loss: 1.6938
Epoch 3/3
938/938 [==============================] - 2s 3ms/step - loss: 1.6827
